The goal of this notebook is to compare two populations...one sampled from the NHANES dataset directly and one sampled from the Gaussian distributions which were in turn obtained from the NHANES dataset...hence the goal is to validate the Gaussian distributions we obtained...

First we read the df with the person information as obtained from the Gaussian distributions, then we create Person-objects, and then the population object. Finally we compare this population and the NHANESDirectSamplePopulation in baseline characteristics, characteristics over time, and cv events.

In [1]:
import os
import copy
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
from scipy.stats import multivariate_normal

from microsim.population import (NHANESDirectSamplePopulation, 
                                 build_people_using_nhanes_for_sampling,
                                 microsimToNhanes, Population, get_nhanes_population)
from microsim.risk_model_repository import RiskModelRepository
from microsim.gender import NHANESGender
from microsim.smoking_status import SmokingStatus
from microsim.race_ethnicity import NHANESRaceEthnicity
from microsim.education import Education
from microsim.treatment import DefaultTreatmentsType,  TreatmentStrategiesType
from microsim.risk_factor import StaticRiskFactorsType, DynamicRiskFactorsType
from microsim.afib_model import AFibPrevalenceModel
from microsim.pvd_model import PVDPrevalenceModel
from microsim.outcome import Outcome, OutcomeType
from microsim.person import Person
from microsim.population_model_repository import PopulationModelRepository
from microsim.cohort_risk_model_repository import (CohortStaticRiskFactorModelRepository,
                                                   CohortDynamicRiskFactorModelRepository,
                                                   CohortDefaultTreatmentModelRepository)
from microsim.outcome_model_repository import OutcomeModelRepository

microsimDir = "/Users/deligkaris.1/OneDrive - The Ohio State University Wexner Medical Center/MICROSIM"
os.chdir(microsimDir+"/CODE/microsim")

dataDir = microsimDir+"/NOTEBOOKS/DATA"

popSize = 100000

In [2]:
categoricalVars = ["gender", "smokingStatus", "raceEthnicity", "statin",'education',
                  'alcoholPerWeek','anyPhysicalActivity','antiHypertensiveCount']
continuousVars = ['age', 'hdl', 'bmi', 'totChol', 'trig', 'a1c', 'ldl', 'waist', 'creatinine', 'sbp', 'dbp']

In [3]:
nhanesDf = pd.read_stata(microsimDir + "/CODE/microsim/microsim/data/fullyImputedDataset.dta")
nhanesDf = nhanesDf.rename(columns={"level_0":"name"})

In [4]:
df = pd.read_csv(dataDir+"/nhanes-as-gaussians-02.csv")
#df = pd.read_csv(dataDir+"/nhanes-persons-from-uniforms.csv")
df.head()

,name,gender,smokingStatus,raceEthnicity,statin,education,alcoholPerWeek,anyPhysicalActivity,antiHypertensiveCount,age,hdl,bmi,totChol,trig,a1c,ldl,waist,creatinine,sbp,dbp
0,41909,1,0,1,False,1,0,0,0.0,54,41.167309,25.694482,214.167769,210.549305,5.389349,148.330353,102.545480,0.688871,131.688014,78.255367
1,42688,1,0,1,False,1,0,0,0.0,51,46.384726,31.935565,164.732679,110.189826,5.286215,81.605992,112.511680,0.762213,137.004879,74.640230
2,43025,1,0,1,False,1,0,0,0.0,36,47.408049,28.315046,260.486802,126.640687,5.668748,177.731299,99.632114,0.857567,121.974670,90.400776
3,43390,1,0,1,False,1,0,0,0.0,40,45.379713,30.468159,215.360517,117.954243,5.686785,135.276734,103.727511,0.715434,113.273055,70.062711
4,44501,1,0,1,False,1,0,0,0.0,74,53.576906,22.736810,223.833927,63.617831,6.194046,170.082194,95.710354,0.696987,123.342909,81.288198


In [5]:
df["age"].describe()

count    5448.000000
mean       44.689611
std        16.418813
min        16.000000
25%        32.000000
50%        43.000000
75%        56.000000
max        93.000000
Name: age, dtype: float64

In [6]:
df.shape

(5448, 20)

In [7]:
#bring the original nhanes weights because these will help me do the sampling
df = df.merge(nhanesDf[["name","WTINT2YR"]], on="name", how="inner").copy()
df.shape

(5448, 21)

In [8]:
def build_person(x, initializationModelRepository):
    """Takes all Person-instance-related data via x and initializationModelRepository and organizes it,
       passes the organized data to the Person class and returns a Person instance."""
    
    rng = np.random.default_rng()
        
    name = x.name
    
    personStaticRiskFactors = {
                        StaticRiskFactorsType.RACE_ETHNICITY.value: NHANESRaceEthnicity(int(x.raceEthnicity)),
                        StaticRiskFactorsType.EDUCATION.value: Education(int(x.education)),
                        StaticRiskFactorsType.GENDER.value: NHANESGender(int(x.gender)),
                        StaticRiskFactorsType.SMOKING_STATUS.value: SmokingStatus(int(x.smokingStatus))}
        
    #use this to get the bounds imposed on the risk factors in a bit
    rfRepository = RiskModelRepository()
        
    #TO DO: find a way to include everything here, including the rfs that need initialization
    #the PVD model would be easy to implement, eg with an estimate_next_risk_for_patient_characteristics function
    #but the AFIB model would be more difficult because it relies on statsmodel_logistic_risk file
    #for now include None, in order to create the risk factor lists correctly at the Person instance
    personDynamicRiskFactors = dict()
    for rfd in DynamicRiskFactorsType:
        #if rfd==DynamicRiskFactorsType.ALCOHOL_PER_WEEK:
        #    personDynamicRiskFactors[rfd.value] = AlcoholCategory.get_category_for_consumption(rfRepository.apply_bounds(rfd.value, x[microsimToNhanes[rfd.value]]))
        #else:
        if (rfd!=DynamicRiskFactorsType.PVD) & (rfd!=DynamicRiskFactorsType.AFIB):
            personDynamicRiskFactors[rfd.value] = rfRepository.apply_bounds(rfd.value, x[rfd.value])
    personDynamicRiskFactors[DynamicRiskFactorsType.AFIB.value] = None
    personDynamicRiskFactors[DynamicRiskFactorsType.PVD.value] = None

    #Q: do we need otherLipid treatment? I am not bringing it to the Person objects for now.
    #A: it is ok to leave it out as we do not have a model to update this. It is also very rarely taking place in the population anyway.
    #also: used to have round(x.statin) but NHANES includes statin=2...
    personDefaultTreatments = {
                        DefaultTreatmentsType.STATIN.value: bool(x.statin),
                        #DefaultTreatmentsType.OTHER_LIPID_LOWERING_MEDICATION_COUNT.value: x.otherLipidLowering,
                        DefaultTreatmentsType.ANTI_HYPERTENSIVE_COUNT.value: x.antiHypertensiveCount}

    personTreatmentStrategies = dict(zip([strategy.value for strategy in TreatmentStrategiesType],
                                         #[None for strategy in range(len(TreatmentStrategiesType))]))
                                         [{"status": None} for strategy in range(len(TreatmentStrategiesType))]))

    personOutcomes = dict(zip([outcome for outcome in OutcomeType],
                                  [list() for outcome in range(len(OutcomeType))]))
    #add pre-simulation stroke outcomes
    #selfReportStrokeAge=x.selfReportStrokeAge
    #Q: we should not add the stroke outcome in case of "else"?
    #if selfReportStrokeAge is not None and selfReportStrokeAge > 1:
    #        selfReportStrokeAge = selfReportStrokeAge if selfReportStrokeAge <= x.age else x.age
    #        personOutcomes[OutcomeType.STROKE].append((selfReportStrokeAge, StrokeOutcome(False, None, None, None, priorToSim=True)))
    #add pre-simulation mi outcomes
    #selfReportMIAge=rng.integers(18, x.age) if x.selfReportMIAge == 99999 else x.selfReportMIAge
    #if selfReportMIAge is not None and selfReportMIAge > 1:
    #        selfReportMIAge = selfReportMIAge if selfReportMIAge <= x.age else x.age
    #        personOutcomes[OutcomeType.MI].append((selfReportMIAge, Outcome(OutcomeType.MI, False, priorToSim=True)))

    person = Person(name,
                   personStaticRiskFactors,
                   personDynamicRiskFactors,
                   personDefaultTreatments,
                   personTreatmentStrategies,
                   personOutcomes)

    #TO DO: find a way to initialize these rfs above with everything else
    person._pvd = [initializationModelRepository[DynamicRiskFactorsType.PVD].estimate_next_risk(person)]
    person._afib = [initializationModelRepository[DynamicRiskFactorsType.AFIB].estimate_next_risk(person)]
    return person


In [9]:
initializationModelRepository = {DynamicRiskFactorsType.AFIB: AFibPrevalenceModel(), 
                                     DynamicRiskFactorsType.PVD: PVDPrevalenceModel()}

In [10]:
dfSample = df.sample(popSize, weights=df.WTINT2YR, replace=True)

In [11]:
people = dfSample.apply(lambda x: build_person(x, initializationModelRepository), axis=1)

In [12]:
#sets the unique identifier for each Person instance
noneList = list(map(lambda person, i: setattr(person, "_index", i), people, range(len(people)))) 

In [13]:
popModelRepository = PopulationModelRepository(CohortDynamicRiskFactorModelRepository(),
                                                           CohortDefaultTreatmentModelRepository(),
                                                           OutcomeModelRepository(),
                                                           CohortStaticRiskFactorModelRepository()) 
pop = Population(people, popModelRepository)

In [14]:
pop._people.shape

(100000,)

In [15]:
#pop2 = get_nhanes_population(1999)
pop2 = NHANESDirectSamplePopulation(popSize, 1999)
pop2.advance(1)
#pop2.print_baseline_summary()

In [16]:
pop.print_baseline_summary_comparison(pop2)

                                 self                                             other
                                 min    0.25   med    0.75    max   mean    sd    min    0.25   med    0.75    max   mean    sd
                           age   18.0   33.0   43.0   55.0   93.0   44.7   15.4   18.0   30.0   41.0   55.0   85.0   43.7   17.2
                           sbp   77.4  112.8  122.1  133.8  221.1  124.0   16.7   72.7  109.3  119.3  131.3  266.0  122.6   19.0
                           dbp   37.9   65.7   72.3   78.8  121.9   72.4    9.8   40.0   65.3   72.0   79.3  132.0   72.4   11.2
                           a1c    3.7    5.0    5.2    5.6   10.1    5.4    0.6    2.5    5.0    5.2    5.5   15.1    5.4    0.9
                           hdl   18.0   42.4   51.3   60.5  114.1   52.2   13.4    8.0   40.0   48.0   59.0  151.0   50.9   15.7
                           ldl   39.9  106.5  127.1  148.4  238.7  128.1   30.7   28.0   97.0  120.0  144.0  344.0  122.9   36.9
          

In [17]:
popages = pop.get_age_of_all_years_in_sim()
popageCounts = pop.get_age_counts(popages)
popages2 = pop2.get_age_of_all_years_in_sim()
popageCount2s = pop2.get_age_counts(popages2)

print("age pop-count pop2-count %diff") 
for key in sorted(popageCounts):
    if key in popageCount2s:
        pcDiff =  (popageCounts[key] - popageCount2s[key])/popageCount2s[key]
        print(f"{key} {popageCounts[key]:>6.0f} {popageCount2s[key]:>10.0f} {pcDiff:>9.2f}")
    else:
        print(f"{key} {popageCounts[key]:>6.0f}")

age pop-count pop2-count %diff
18   1392       1756     -0.21
19    631       2044     -0.69
20    774       1983     -0.61
21   1262       2299     -0.45
22   1166       2428     -0.52
23   1416       1920     -0.26
24   1490       1836     -0.19
25   1908       1654      0.15
26   1865       1918     -0.03
27   1641       1513      0.08
28   2198       2120      0.04
29   1885       2286     -0.18
30   2117       1994      0.06
31   2021       2079     -0.03
32   2556       2637     -0.03
33   1918       1936     -0.01
34   2830       2075      0.36
35   2143       2521     -0.15
36   2483       2030      0.22
37   2972       2558      0.16
38   1778       2165     -0.18
39   2856       2701      0.06
40   2644       2278      0.16
41   2303       2034      0.13
42   2625       2037      0.29
43   2800       1784      0.57
44   3014       2105      0.43
45   1698       2071     -0.18
46   2450       2240      0.09
47   2211       1743      0.27
48   1805       1523      0.19
49   207

In [ ]:
#personsOut = list(filter( lambda x: ((x._sbp[0]>=72.) & (x._sbp[0]<=81.)) | ((x._sbp[0]>=207.) & (x._sbp[0]<=266.)) , pop2._people))
#len(personsOut)

In [ ]:
#personsOutUnique = set(map( lambda x: x._name, personsOut) )
#len(personsOutUnique)

In [ ]:
%%time
pop.advance(17, None, nWorkers=5)

In [ ]:
%%time
pop2.advance(17, None, nWorkers=5)

In [ ]:
pop.print_lastyear_summary_comparison(pop2)

In [ ]:
%%time
pop.print_cv_standardized_rates()

In [ ]:
%%time
pop2.print_cv_standardized_rates()

In [ ]:
%%time
pop.print_dementia_incidence()

In [ ]:
nYears=5
nSimulations = 4
for bpMedsAdded in [1]:
    miRRList = list()
    strokeRRList = list()
    print(f"\nbpMedsAdded={bpMedsAdded}")
    for i in range(nSimulations):
        treatmentPop = NHANESDirectSamplePopulation(popSize, 1999)
        controlPop = NHANESDirectSamplePopulation(popSize, 1999)
        treatmentStrategies = TreatmentStrategyRepository()
        #treatmentStrategies._repository[TreatmentStrategiesType.BP.value] = AddASingleBPMedTreatmentStrategy()
        treatmentStrategies._repository[TreatmentStrategiesType.BP.value] = AddNBPMedsTreatmentStrategy(bpMedsAdded)
        treatmentPop.advance_parallel(1, treatmentStrategies = treatmentStrategies, nWorkers=nWorkers)
        treatmentStrategies._repository[TreatmentStrategiesType.BP.value].status = TreatmentStrategyStatus.MAINTAIN
        treatmentPop.advance_parallel(nYears-1, treatmentStrategies = treatmentStrategies, nWorkers=nWorkers)
        controlPop.advance_parallel(nYears, nWorkers=nWorkers)
        controlRisk = sum(list(map(lambda x: x.has_outcome_during_simulation(OutcomeType.STROKE),
                                       controlPop._people)))/controlPop._n
        treatmentRisk = sum(list(map(lambda x: x.has_outcome_during_simulation(OutcomeType.STROKE),
                                         treatmentPop._people)))/treatmentPop._n
        strokeRR = treatmentRisk/controlRisk
        controlRisk = sum(list(map(lambda x: x.has_outcome_during_simulation(OutcomeType.MI),
                                       controlPop._people)))/controlPop._n
        treatmentRisk = sum(list(map(lambda x: x.has_outcome_during_simulation(OutcomeType.MI),
                                         treatmentPop._people)))/treatmentPop._n
        miRR = treatmentRisk/controlRisk
        miRRList += [miRR]
        strokeRRList += [strokeRR]
        print(f"\t\tsimulation={i}, stroke RR={strokeRR:<4.2f}, mi RR={miRR:<4.2f}")
    print(f"average of {nSimulations} simulations: stroke RR={np.mean(strokeRRList):<4.2f}, mi RR={np.mean(miRRList):4.2f}")
    print(f"sd of {nSimulations} simulations: stroke RR={np.std(strokeRRList):<4.2f}, mi RR={np.std(miRRList):<4.2f}")
